In [ ]:
import pandas as pd
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import gspread
import os

def connect_to_sheets(credentials_file):
    credentials = service_account.Credentials.from_service_account_file(
        credentials_file,
        scopes=['https://www.googleapis.com/auth/spreadsheets']
    )
    
    gc = gspread.authorize(credentials)
    spreadsheet_url = os.getenv('SPREADSHEET_URL')
    return gc.open_by_url(spreadsheet_url)

def read_worksheet_to_df(spreadsheet, worksheet_name):
    worksheet = spreadsheet.worksheet(worksheet_name)
    all_values = worksheet.get_all_values()
    headers = all_values[0]
    data = all_values[1:]
    return pd.DataFrame(data, columns=headers)

def separate_opening_stock(df):
    opening_stock_mask = df['purchasing_officer'].str.contains('opening stock', case=False, na=False)
    opening_stock_df = df[opening_stock_mask].copy()
    main_df = df[~opening_stock_mask].copy()
    return main_df, opening_stock_df

def standardize_dataframe(df):
    df_clean = df.copy()
    
    df_clean.columns = df_clean.columns.str.lower()  
    df_clean.columns = df_clean.columns.str.strip()
    df_clean.columns = df_clean.columns.str.replace(' ', '_')  
    df_clean.columns = df_clean.columns.str.replace('-', '_')
    
    for column in df_clean.columns:
        df_clean[column] = df_clean[column].astype(str)
        df_clean[column] = df_clean[column].str.strip().str.lower()
        
        try:
            numeric_values = df_clean[column].str.replace(',', '').astype(float)
            df_clean[column] = numeric_values
        except (ValueError, TypeError):
            pass
    
    return df_clean

def standardize_dates(df):
    if df.empty:
        return df
        
    df = df.copy()
    
    try:
        df['date'] = pd.to_datetime(df['date'], format='%d %b %Y')
    except ValueError:
        try:
            df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')
        except ValueError:
            df['date'] = pd.to_datetime(df['date'], format='mixed', dayfirst=True)
    
    df['month'] = df['date'].dt.strftime('%b').str.lower()
    df['year_month'] = df['date'].dt.strftime('%Y-%b')
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    return df

def create_summary_df(stock_inflow_df, release_df):
    summary_df = pd.DataFrame()
    
    # Get unique months from both dataframes
    inflow_months = set(stock_inflow_df['month'].unique())
    release_months = set(release_df['month'].unique())
    all_months = sorted(list(inflow_months.union(release_months)))
    
    # Get unique year_months from both dataframes
    inflow_year_months = set(stock_inflow_df['year_month'].unique())
    release_year_months = set(release_df['year_month'].unique())
    all_year_months = sorted(list(inflow_year_months.union(release_year_months)))
    
    # Create base dataframe with all months
    summary_df = pd.DataFrame({
        'month': all_months,
        'year_month': all_year_months
    })
    
    # Calculate chicken inflow summaries
    chicken_inflow = stock_inflow_df[stock_inflow_df['product_type'] == 'chicken'].groupby('month').agg({
        'quantity': 'sum',
        'weight': 'sum'
    }).reset_index()
    
    # Calculate chicken release summaries
    chicken_release = release_df[release_df['product'] == 'chicken'].groupby('month').agg({
        'quantity': 'sum',
        'weight_in_kg': 'sum'
    }).reset_index()
    
    # Calculate gizzard inflow summaries
    gizzard_inflow = stock_inflow_df[stock_inflow_df['product_type'] == 'gizzard'].groupby('month').agg({
        'weight': 'sum'
    }).reset_index()
    
    # Calculate gizzard release summaries
    gizzard_release = release_df[release_df['product'] == 'gizzard'].groupby('month').agg({
        'weight_in_kg': 'sum'
    }).reset_index()
    
    # Merge all data with the summary dataframe
    summary_df = summary_df.merge(chicken_inflow, on='month', how='left')
    summary_df = summary_df.merge(
        chicken_release.rename(columns={
            'quantity': 'total_chicken_release_quantity',
            'weight_in_kg': 'total_chicken_release_weight'
        }), 
        on='month', 
        how='left'
    )
    
    summary_df = summary_df.merge(
        gizzard_inflow.rename(columns={'weight': 'total_gizzard_inflow_weight'}),
        on='month',
        how='left'
    )
    
    summary_df = summary_df.merge(
        gizzard_release.rename(columns={'weight_in_kg': 'total_gizzard_release_weight'}),
        on='month',
        how='left'
    )
    
    # Rename columns
    summary_df = summary_df.rename(columns={
        'quantity': 'total_chicken_inflow_quantity',
        'weight': 'total_chicken_inflow_weight'
    })
    
    # Fill NaN values with 0
    numeric_columns = summary_df.select_dtypes(include=['float64', 'int64']).columns
    summary_df[numeric_columns] = summary_df[numeric_columns].fillna(0)
    
    # Round numeric columns to 2 decimal places
    summary_df[numeric_columns] = summary_df[numeric_columns].round(2)
    
    # Reorder columns to match desired output
    column_order = [
        'month',
        'year_month',
        'total_chicken_inflow_quantity',
        'total_chicken_inflow_weight',
        'total_chicken_release_quantity',
        'total_chicken_release_weight',
        'total_gizzard_inflow_weight',
        'total_gizzard_release_weight'
    ]
    
    summary_df = summary_df[column_order]
    
    return summary_df

def process_sheets_data(stock_inflow_df, release_df):
    stock_inflow_df = standardize_dataframe(stock_inflow_df)
    release_df = standardize_dataframe(release_df)
    
    stock_inflow_main_df, opening_stock_df = separate_opening_stock(stock_inflow_df)
    
    release_df = release_df[~release_df['name_of_collector'].str.contains('opening stock', case=False, na=False)]
    
    # Set quantity to 0 for gizzard products
    release_df.loc[release_df['product'].str.contains('gizzard', case=False, na=False), 'quantity'] = 0
    
    stock_inflow_main_df = standardize_dates(stock_inflow_main_df)
    opening_stock_df = standardize_dates(opening_stock_df)
    release_df = standardize_dates(release_df)
    
    summary_df = create_summary_df(stock_inflow_main_df, release_df)
    
    return stock_inflow_main_df, opening_stock_df, release_df, summary_df

def upload_df_to_gsheet(df, spreadsheet_id, sheet_name, credentials_file):
    try:
        df_copy = df.copy()
        
        # Format datetime columns
        datetime_columns = df_copy.select_dtypes(include=['datetime64[ns]']).columns
        for col in datetime_columns:
            df_copy[col] = df_copy[col].dt.strftime('%Y-%m-%d')
        
        # Ensure numeric columns are formatted properly
        numeric_columns = df_copy.select_dtypes(include=['float64', 'int64']).columns
        for col in numeric_columns:
            df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce').fillna(0)
        
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
        credentials = service_account.Credentials.from_service_account_file(
            credentials_file,
            scopes=SCOPES
        )
        
        service = build('sheets', 'v4', credentials=credentials)
        
        values = [df_copy.columns.values.tolist()]
        values.extend(df_copy.values.tolist())
        
        body = {
            'values': values
        }
        
        # Clear existing content
        clear_request = service.spreadsheets().values().clear(
            spreadsheetId=spreadsheet_id,
            range=f'{sheet_name}!A1:ZZ'
        )
        clear_request.execute()
        
        # Update with new content
        result = service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id,
            range=f'{sheet_name}!A1',
            valueInputOption='USER_ENTERED',
            body=body
        ).execute()
        
        # Get sheet ID
        sheet_metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheet_id = None
        for sheet in sheet_metadata.get('sheets', ''):
            if sheet.get('properties', {}).get('title') == sheet_name:
                sheet_id = sheet.get('properties', {}).get('sheetId')
                break
        
        if sheet_id is not None:
            requests = [{
                'repeatCell': {
                    'range': {
                        'sheetId': sheet_id,
                        'startRowIndex': 1,
                        'startColumnIndex': 2,  # Start from the first numeric column
                        'endColumnIndex': len(df_copy.columns)
                    },
                    'cell': {
                        'userEnteredFormat': {
                            'numberFormat': {
                                'type': 'NUMBER',
                                'pattern': '#,##0.00'
                            }
                        }
                    },
                    'fields': 'userEnteredFormat.numberFormat'
                }
            }]
            
            # Apply the formatting
            format_body = {'requests': requests}
            service.spreadsheets().batchUpdate(
                spreadsheetId=spreadsheet_id,
                body=format_body
            ).execute()
        
        print(f"Updated {result.get('updatedCells')} cells in {sheet_name}")
        return True
        
    except HttpError as error:
        print(f"An error occurred in {sheet_name}: {error}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred in {sheet_name}: {str(e)}")
        return False

def main():
    CREDENTIALS_FILE = 'credentials.json' 
    
    try:
        spreadsheet = connect_to_sheets(CREDENTIALS_FILE)
        
        stock_inflow_df = read_worksheet_to_df(spreadsheet, 'stock_inflow')
        release_df = read_worksheet_to_df(spreadsheet, 'release')
        
        stock_inflow_main_df, opening_stock_df, release_df, summary_df = process_sheets_data(stock_inflow_df, release_df)
        
        spreadsheet_id = os.getenv('SPREADSHEET_ID')
        
        success_stock = upload_df_to_gsheet(
            stock_inflow_main_df, 
            spreadsheet_id, 
            'stock_inflow_clean',
            CREDENTIALS_FILE
        )
        
        success_opening_stock = upload_df_to_gsheet(
            opening_stock_df, 
            spreadsheet_id, 
            'opening_stock',
            CREDENTIALS_FILE
        )
        
        success_release = upload_df_to_gsheet(
            release_df, 
            spreadsheet_id, 
            'release_clean',
            CREDENTIALS_FILE
        )
        
        success_summary = upload_df_to_gsheet(
            summary_df,
            spreadsheet_id,
            'summary',
            CREDENTIALS_FILE
        )
        
        if success_stock and success_opening_stock and success_release and success_summary:
            print("Data processing and upload completed successfully!")
        else:
            raise Exception("Failed to upload one or more datasets")
            
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

if __name__ == "__main__":
    main()